## Imports

In [1]:
import numpy as np
import linecache as lc

# Aggregating postcodes by alphabetical prefix

In [2]:
customer_id_og = lc.getline(r"CaseStudyData.txt", 17).split(" ")[2:-1]
customer_id_og = [id.strip('"') for id in customer_id_og]

candidate_id_og = lc.getline(r"CaseStudyData.txt", 27).split(" ")[2:-1]
candidate_id_og = [id.strip('"') for id in candidate_id_og]

In [3]:
def extract_first_occurrence(postcodes_list):
    first_occurrences = {}
    
    for i, postcode in enumerate(postcodes_list):
        prefix = postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else (postcode[0] if postcode and postcode[0].isalpha() else '')
        # prefix = postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else (postcode[0] if postcode[0].isalpha() else '')
        if prefix not in first_occurrences:
            first_occurrences[prefix] = {'postcode': postcode, 'index': i}
    
    unique_prefixes = list(first_occurrences.keys())
    first_postcodes = [first_occurrences[prefix]['postcode'] for prefix in unique_prefixes]
    first_indices = [first_occurrences[prefix]['index'] for prefix in unique_prefixes]
    
    return unique_prefixes, first_postcodes, first_indices

customer_id_selective = extract_first_occurrence(customer_id_og)
candidate_id_selective = extract_first_occurrence(candidate_id_og)

# Aggregating the 1D vectors

In [4]:
def aggregate_postcodes(list):
    
    # Extract unique prefixes
    unique_prefixes = np.unique([postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else postcode[0] for postcode in list])

    # Create array with prefixes
    aggregated_values = np.array([postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else postcode[0] for postcode in list])

    return unique_prefixes, aggregated_values

In [5]:
def process_aggregated_values(postcode_list, value_list, operation='average'):
    
    '''
    postcode_list: the reference data to obtain the unique postcode prefixes from
    value_list: the vector we wish to aggregate/group
    operation: can choose between averging, summing, or taking the maximum value when aggregating
    
    This function takes the arguments listed above and returns a vector of 
    aggregated values based on the operation chosen.
    '''
    
    unique_prefixes, aggregated_values = aggregate_postcodes(postcode_list)
    
    values_dict = {prefix: [] for prefix in unique_prefixes}

    for prefix, value in zip(aggregated_values, value_list):
        values_dict[prefix].append(value)

    if operation == 'average':
        result_array = np.array([np.mean(values_dict[prefix]) for prefix in unique_prefixes])
    elif operation == 'sum':
        result_array = np.array([np.sum(values_dict[prefix]) for prefix in unique_prefixes])
    elif operation == 'maximum':
        result_array = np.array([np.max(values_dict[prefix]) for prefix in unique_prefixes])
    else:
        raise ValueError("Invalid operation. Please choose 'average', 'sum', or 'maximum'.")

    return result_array

In [6]:
def extract_selected_values(postcode_list, value_list):
    first_indices = extract_first_occurrence(postcode_list)[2]
    return [value_list[i] for i in first_indices]

## Aggregating coordinates

In [7]:
customer_easting = lc.getline(r"CaseStudyData.txt", 20).split(" ")[2:-1]
customer_easting = [eval(coord) for coord in customer_easting]
customer_easting_selective = extract_selected_values(customer_id_og, customer_easting)

customer_northing = lc.getline(r"CaseStudyData.txt", 21).split(" ")[2:-1]
customer_northing = [eval(coord) for coord in customer_northing]
customer_northing_selective = extract_selected_values(customer_id_og, customer_northing)

candidate_easting = lc.getline(r"CaseStudyData.txt", 30).split(" ")[2:-1]
candidate_easting = [eval(coord) for coord in candidate_easting]
candidate_easting_selective = extract_selected_values(candidate_id_og, candidate_easting)

candidate_northing = lc.getline(r"CaseStudyData.txt", 31).split(" ")[2:-1]
candidate_northing = [eval(coord) for coord in candidate_northing]
candidate_northing_selective = extract_selected_values(candidate_id_og, candidate_northing)

## Aggregating  warehouse costs and capacities

In [8]:
def process_costs_capacities(line_start, line_stop):
    
    '''
    Simply takes the line to start reading, and the line to stop reading. Then, it 
    processes and transforms the 1D cost and capacity vectors into a format that we can use.
    '''
    
    file = open(r"CaseStudyData.txt", "r")
    costs_og = file.readlines()[line_start-1:line_stop]
    costs_og = list(map(lambda s: s.strip(), costs_og))

    costs_og = [costs_og[line].split(" ") for line in range(len(costs_og))]
    costs = []
    for line in costs_og:
        for i in range(len(line)):
            costs.append(line[i].strip('[]'))
            
    costs = costs[2:]        
    costs = [eval(cost) for cost in costs]
    return costs

In [9]:
setup_costs = process_costs_capacities(62, 105)
operating_costs = process_costs_capacities(108, 146)
wh_capacities = process_costs_capacities(149, 193)

setup_costs_selective = extract_selected_values(customer_id_og, setup_costs)
operating_costs_selective = extract_selected_values(customer_id_og, operating_costs)
wh_capacities_selective = extract_selected_values(customer_id_og, wh_capacities)

In [10]:
customer_populations = lc.getline(r"CaseStudyData.txt", 24).split(" ")[2:-1]
customer_populations = [eval(coord) for coord in customer_populations]
customer_populations_selective = extract_selected_values(customer_id_og, customer_populations)

# Aggregating the 2D arrays

We start off with 4 $\times$ 440 = 1760 values, so we expect to be left with 4 vectors in the end - one for each product type.

In [11]:
def process_nD(line_start, line_stop, no_to_delete = None):
    
    '''
    Very similar to the function for processing costs and capacities, but this 
    one doesn't automatically remove the first two elements from the beginning like
    the other one. This is because the formatting for 2D stuff is a little different 
    and requires some further processing outside the function too.
    
    The logic is to basically get it all cleaned up within the function, then 
    reshape and delete columns outside.
    '''
    
    file = open(r"CaseStudyData.txt", "r")
    list_og = file.readlines()[line_start-1:line_stop]
    list_og = list(map(lambda s: s.strip(), list_og))

    list_og = [list_og[line].split(" ") for line in range(len(list_og))]
    processed_list = []
    for line in list_og:
        for i in range(len(line)):
            processed_list.append(line[i].strip('[]'))
    
    if no_to_delete != None:
        processed_list = processed_list[no_to_delete:]
    else:
        pass
    return processed_list

In [12]:
demand_per_product = np.asarray(process_nD(197, 343)).reshape(440, 6)[:, 2:]
demand_per_product = demand_per_product.astype(int)

dpp_selective = []
for i in range(4):
    dpp_selective.append(process_aggregated_values(customer_id_og, demand_per_product[:,i], "sum"))

dpp_selective = np.asarray(dpp_selective)

# Aggregating the 3D array

In [13]:
dpp_per_year = np.asarray(process_nD(347, 1693, 1)).reshape(440, 4, -1)[:, :, 3:]
dpp_per_year = dpp_per_year.astype(int)

dpp_py_selective = []
for i in range(4):
    dpp_sel = []
    for j in range(10):
        dpp_sel.append(process_aggregated_values(customer_id_og, dpp_per_year[:, i, j], "sum"))
    dpp_py_selective.append(dpp_sel)
dpp_py_selective = np.asarray(dpp_py_selective)
dpp_py_selective[0].shape

(10, 15)

# Aggregating the 4D array

In [14]:
dppy_scenarios = np.asarray(process_nD(1695, 115666, 1)).reshape(440, 4, 10, -1)[:, :, :, 4:]
dppy_scenarios  = dppy_scenarios.astype(int)

dppys_selective = []
for i in range(4):
    dppy_sel = []
    for j in range(10):
        dp_sel = []
        for k in range(100):
            dp_sel.append(process_aggregated_values(customer_id_og, dppy_scenarios[:, i, j, k], "sum"))
        dppy_sel.append(dp_sel)
    dppys_selective.append(dppy_sel)
    
dppys_selective = np.asarray(dppys_selective)

In [15]:
can_supp_distances = np.asarray(process_nD(115670, 116109)).reshape(440, -1)[:, 2:]
can_supp_distances  = can_supp_distances.astype(float)

csd_selective = []
for i in range(53):
    csd_selective.append(extract_selected_values(customer_id_og, can_supp_distances[:,i]))
csd_selective = np.asarray(csd_selective)

In [16]:
can_cust_distances = np.asarray(process_nD(116114, 116553)).reshape(440, -1)[:, 2:]
can_cust_distances  = can_cust_distances.astype(float)

ccd_selective = []
for i in range(53):
    ccd_selective.append(extract_selected_values(customer_id_og, can_cust_distances[:,i]))
ccd_selective = np.asarray(ccd_selective)

In [17]:
csd_selective

array([[192.61, 180.02, 265.51, 209.66, 211.69, 189.6 , 106.26,  87.6 ,
        205.3 , 170.09, 192.38, 202.55, 182.51, 163.93, 246.09],
       [175.97, 112.58, 131.58, 101.83,  73.99,  55.68, 269.55, 147.24,
         71.38, 247.49, 105.11,  68.63,  48.59,  91.21, 132.23],
       [179.64, 147.99, 218.67, 177.63, 147.48, 142.76, 220.07,  97.76,
        158.47, 198.01, 160.35, 155.72, 135.67, 131.9 , 214.06],
       [ 18.06,  56.74, 203.08, 111.65, 116.74, 135.83, 288.81,  96.37,
        159.86, 198.62,  84.62, 133.24, 144.99,  76.01, 148.08],
       [141.07,  77.68, 105.62,  65.06,  44.53,  29.71, 277.58, 166.23,
         45.42, 268.48,  70.2 ,  42.67,  22.62,  56.3 ,  97.75],
       [190.36, 110.06,  25.49,  52.95,  71.06,  57.91, 350.65, 215.52,
         61.03, 317.78,  83.7 ,  47.15,  64.11, 105.6 ,  42.71],
       [ 50.89,  18.03, 165.12,  72.84,  78.95,  97.87, 303.85, 132.69,
        121.9 , 234.94,  45.81,  94.43, 107.03,  44.62, 109.27],
       [179.24, 115.41,  95.71,  54.  ,  

In [18]:
# product, year, scenario, customer - in python


# customer, product, year, scenario - in xpress


# Writing to a txt file

In [19]:
# Strings to be written before each array
# File path
output_file = f"CaseStudyData_Aggregated.txt"

# Writing to the file
with open(output_file, 'w') as file:
    # Write number of customers, locations
    file.write("! Number of customers = postcode districts\n")
    file.write(f"nbCustomers: {len(customer_id_selective[0])}\n")
    file.write("! Number of candidate locations\n")
    file.write(f"nbCandidates: {len(candidate_id_selective[0])}\n \n")
    
    # file.write("\n")


    # Write Customer IDs
    file.write("! Vector of customer ids\n")
    customer_id_output = "CustomerId: [ " + " ".join('"' + s + '"' for s in customer_id_selective[0]) + " ]\n"
    file.write(customer_id_output)

    # Write Customer eastings and northings
    file.write("\n! Vector of customer coordinates\n")
    file.write("CustomerEasting: [" )
    np.savetxt(file, customer_easting_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n")
    file.write("CustomerNorthing: [" )
    np.savetxt(file, customer_northing_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")

    # Write Customer populations
    file.write("! Vector of customer populations\nCustomerPopulation: [ ")
    np.savetxt(file, customer_populations_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")

    # Write Candidate IDs
    file.write("! Vector of candidate location ids\n")
    candidate_id_output = "CandidateId: [ " + " ".join('"' + s + '"' for s in candidate_id_selective[0]) + " ]\n"
    file.write(candidate_id_output)

    # Write Customer eastings and northings
    file.write("\n! Vector of candidate location coordinates\n")
    file.write("CandidateEasting: [" )
    np.savetxt(file, candidate_easting_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n")
    file.write("CandidateNorthing: [" )
    np.savetxt(file, candidate_northing_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")

    # Write set up and operating costs
    file.write("! Setup cost for warehouses\nSetup: [(1) ")
    np.savetxt(file, setup_costs_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")
    file.write("! Operating cost for warehouses\nOperating: [(1) ")
    np.savetxt(file, operating_costs_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")
    file.write("! The warehouse capacity\nCapacity: [(1) ")
    np.savetxt(file, wh_capacities_selective, fmt='%d', delimiter=' ', newline=" ")
    file.write("]\n \n")

    # Write annual district demand in kg per product group
    file.write("! The annual district demand in kilograms per product group\nCustomerDemand: [\n")
    for i in range(len(customer_id_selective[0])):
        file.write(f"({i+1} 1) {dpp_selective[0,i]} {dpp_selective[1,i]} {dpp_selective[2,i]} {dpp_selective[3,i]} \n")
    file.write("]\n \n")

    # Write customer demand in kilograms per product group over nbPeriod years
    file.write("! The customer demand in kilograms per product group over nbPeriods years\nCustomerDemandPeriods: [")
    for cust in range(len(customer_id_selective[0])):
        for pro in range(4):
            file.write(f"\n({cust+1} {pro+1} 1) ")
            for per in range(10):
                file.write(f"{dpp_py_selective[pro, per, cust]} ")
    file.write("]\n \n")

    # Write customer demand per period for each scenario
    file.write("CustomerDemandPeriodScenarios: [")
    for cust in range(len(customer_id_selective[0])):
        for pro in range(4):
            for per in range(10):
                file.write(f"\n({cust+1} {pro+1} {per+1} 1) ")
                for scen in range(100):
                    file.write(f"{dppys_selective[pro, per, scen, cust]} ")
    file.write("] \n \n")


    # Write distance matrix between candidate and supplier
    file.write("! Distance matrix between candidate locations and suppliers\nDistanceCandidateSupplier: [")
    for can in range(len(candidate_id_selective[0])):
        file.write(f"\n({can+1} 1) ")
        for supp in range(53):
            file.write(f"{csd_selective[supp, can]} ")
    file.write("]\n \n")


    # Write distance matrix between customer and candidate
    file.write("! Distance matrix between candidate locations and customers\nDistanceCandidateCustomer: [")
    for can in range(len(candidate_id_selective[0])):
        file.write(f"\n({can+1} 1) ")
        for cust in range(len(customer_id_selective[0])):
            file.write(f"{ccd_selective[cust, can]} ")
    file.write("]\n \n")
        
